In [ ]:
!pip install pycaret

In [ ]:
import pandas as pd
from sklearn.preprocessing import normalize
import numpy as np
import category_encoders as ce
from pycaret.regression import * 

In [ ]:
df_train = pd.read_csv("../input/big-mart-sales-prediction/Train.csv")
df_test = pd.read_csv("../input/big-mart-sales-prediction/Test.csv")
df_test_copy = df_test.copy()

In [ ]:
df_train

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
df_train["Outlet_Size"].value_counts()

In [ ]:
def fillVal(df,column,value):
    df[column].fillna(value,inplace=True)
    return df

In [ ]:
df_train = fillVal(df_train,"Item_Weight",np.mean(df_train["Item_Weight"]))
df_train = fillVal(df_train,"Outlet_Size","Unknown")
df_test = fillVal(df_test,"Item_Weight",np.mean(df_train["Item_Weight"]))
df_test = fillVal(df_test,"Outlet_Size","Unknown")

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train["Item_Fat_Content"].value_counts()

In [ ]:
def one_hot_encoding(df,col):
    one_hot_encoder=ce.OneHotEncoder(cols=col,return_df=True,use_cat_names=True)
    df_final = one_hot_encoder.fit_transform(df)
    return df_final

In [ ]:
for column in df_train.columns:
    if column!="Item_Identifier" and df_train[column].dtypes == "object" or column =="Outlet_Establishment_Year":
        df_train =  one_hot_encoding(df_train,column)

for column in df_test.columns:
    if column!="Item_Identifier" and df_test[column].dtypes == "object" or column =="Outlet_Establishment_Year":
        df_test =  one_hot_encoding(df_test,column)

In [ ]:
def drop_columns(df,columns):
    return df.drop(columns,axis = 1)

In [ ]:
df_train = drop_columns(df_train,"Item_Identifier")
df_test = drop_columns(df_test,"Item_Identifier")

In [ ]:
df_train

In [ ]:
df_test

In [ ]:
X_train = df_train.drop("Item_Outlet_Sales",axis = 1)
X_test = df_test
Y_train = df_train["Item_Outlet_Sales"]

In [ ]:

df_train = pd.DataFrame(X_train,columns = [i for i in df_train.columns if (i!="")])
df_test = pd.DataFrame(X_test,columns = df_test.columns)

In [ ]:
df_train["Item_Outlet_Sales"] = Y_train

In [ ]:
df_train

In [ ]:
experiment = setup(df_train,target="Item_Outlet_Sales")

In [ ]:
compare_models()

In [ ]:
gbr = create_model('gbr')

In [ ]:
Y_pred = predict_model(gbr,data = df_test)

In [ ]:
Y_pred

In [ ]:
submission = pd.DataFrame()
submission[["Item_Identifier","Outlet_Identifier"]] = df_test_copy[["Item_Identifier","Outlet_Identifier"]]
submission["Item_Outlet_Sales"] = abs(Y_pred["Label"])


In [ ]:
submission.to_csv("submission.csv",index = False)